In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function

In [2]:

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = 'IEEE_13Node_Modified_01.dss'
dss.run_command('Redirect ' + fn)

''

Vectorized Solver

In [7]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE13_splitmultiphase.dss'
#fn = 'IEEE13_nominalload_vec.dss'
#fn = 'IEEE13_capnominalvoltage.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'hello.dss'
fn = '02node_threephase_unbalanced_unchanged.dss' #original
#fn = '02node_threephase_unbalanced.dss' #apq = .9, az = .1
#fn = '06node_threephase_unbalanced.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


# Pre-generate matrices
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None)

for i in range(len(times)):
    # Run NR3 with variations in time, DER, and capacitance
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i])    
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
VNR
[[ 1.        +0.00000000e+00j  0.99999713+6.82476515e-06j
   0.99999522+1.11076655e-05j]
 [-0.5       -8.66025404e-01j -0.49999374-8.66029796e-01j
  -0.49999015-8.66029998e-01j]
 [-0.5       +8.66025404e-01j -0.50000686+8.66020704e-01j
  -0.50001002+8.66018889e-01j]]
INR:
[[ 0.22500104-2.99981223e-02j  0.10000048+1.11077718e-06j]
 [-0.1514695 -1.72357139e-01j -0.06682905-1.05753999e-01j]
 [-0.04420054+1.76554010e-01j -0.05000106+8.66019988e-02j]]
STXNR:
[[0.22500104+2.99981223e-02j 0.10000019-4.28294210e-07j]
 [0.22500041+4.49978630e-02j 0.12500022+4.99960682e-03j]
 [0.17500053+4.99982175e-02j 0.1       -3.64325211e-07j]]
SRXNR:
[[0.22500019+0.02999957j 0.1       +0.j        ]
 [0.22500022+0.04499961j 0.125     +0.005j     ]
 [0.175     +0.04999964j 0.1       +0.j        ]]
iNR
[[ 0.        +0.j          0.12500056-0.02999923j  0.1000007 -0.01999898j]
 [ 0.        +0.j         -0.08464045-0.06660314j -0.0

Non-vectorized solver reading OpenDSS files

In [8]:
fn= 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'IEEE13_splitmultiphase.dss'
#fn = 'IEEE13_nominalload_vec.dss'
#fn = 'IEEE13_capnominalvoltage.dss'
#fn = 'hello.dss'
fn = '02node_threephase_unbalanced_unchanged.dss' #original
#fn = '02node_threephase_unbalanced.dss' #aPQ = 0.9 aZ = 0.1
#fn = '06node_threephase_unbalanced.dss' #model = 8
#fn = '06node_threephase_unbalanced.dss' 
#fn = 'IEEE_13Node_Modified_01.dss'

VNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR02 =  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
spah = np.zeros((len(times), 3, nnode), dtype = "complex") #store load values

for i in range(len(times)):
    network1 = nmf2(fn, times[i])

    spah[i, :, :] = network1.loads.spu

    nnode = network1.nodes.nnode
    nline = network1.lines.nline

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    network1.loads.aPQ =1.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aI = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aZ = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)

    network1.loads.spu = 1*network1.loads.spu

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    VNR02[i, :, :] = np.reshape(VNR, (3, nnode))
    INR02[i, :, :] = np.reshape(INR, (3, nline))
    STXNR02[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR02[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR02[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR02[i, :, :] = np.reshape(sNR, (3, nnode))
    print('VNR\n', np.round(VNR,decimals=6))
    print('INR\n', np.round(INR,decimals=6))
    print('STXNR\n', np.round(STXNR,decimals=6))
    print('SRXNR\n', np.round(SRXNR,decimals=6))
    print('iNR\n', np.round(iNR,decimals=6))
    print('sNR\n', np.round(sNR,decimals=6))
    print('itercount:', itercount)

    print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
    print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
    print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

BASE
Vbase: 2886.7513459481293
Sbase: 1000000.0
Ibase: 346.4101615137754
Zbase: 8.333333333333336

NODES
nnode: 3
nodelist: ['sourcebus', 'a01', 'a02']
phases: ['abc', 'abc', 'abc']
PH:
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
LINES
nline: 2
TXnode: ['sourcebus', 'a01']
RXnode: ['a01', 'a02']
TXnum: [0 1]
RXnum: [1 2]
phases: ['abc', 'abc']
PH:
 [[1 1]
 [1 1]
 [1 1]]
config: ['sub_a01', 'a01_a02']
length: [15.24 15.24]

NODES + LINES
inlines:
 [[-1  0  1]]
innodes:
 [[-1  0  1]]
outlines:
 [[ 0  1 -1]]
outnodes:
 [[ 1  2 -1]]

CONFIGS
nconf: 2
conflist: ['sub_a01', 'a01_a02']
sub_a01 - FZpl:
 [[1.11847093e-06+7.45647284e-07j 3.72823642e-07+2.98258914e-06j
  3.72823642e-07+2.98258914e-06j]
 [3.72823642e-07+2.98258914e-06j 1.11847093e-06+7.45647284e-07j
  3.72823642e-07+2.98258914e-06j]
 [3.72823642e-07+2.98258914e-06j 3.72823642e-07+2.98258914e-06j
  1.11847093e-06+7.45647284e-07j]]
a01_a02 - FZpl:
 [[1.11847093e-06+7.45647284e-07j 3.72823642e-07+2.98258914e-06j
  3.72823642e-07+2.98258914e-06j]
 [

egienvalues
1.1837518185582048
0.8447716694686799
3.2469796510874103
0.1980622586627038
egienvalues
1.1837523218267618
0.8447652651840797
3.2980816806458373
0.1855492263777901
egienvalues
1.1837523218320052
0.8447652650265889
3.298081989619835
0.18554922139420038
VNR
 [[ 1.      +0.00000e+00j  0.999997+7.00000e-06j  0.999995+1.20000e-05j]
 [-0.5     -8.66025e-01j -0.499994-8.66030e-01j -0.49999 -8.66030e-01j]
 [-0.5     +8.66025e-01j -0.500007+8.66021e-01j -0.50001 +8.66019e-01j]]
INR
 [[ 0.233655-0.031921j  0.103847-0.000768j]
 [-0.157961-0.178602j -0.070065-0.109437j]
 [-0.045234+0.183729j -0.051258+0.090317j]]
STXNR
 [[0.233655+0.031921j 0.103846+0.000769j]
 [0.233654+0.047498j 0.129808+0.005961j]
 [0.181731+0.05269j  0.103846+0.000769j]]
SRXNR
 [[0.233654+0.031923j 0.103846+0.000769j]
 [0.233654+0.0475j   0.129808+0.005962j]
 [0.181731+0.052692j 0.103846+0.000769j]]
iNR
 [[ 0.      +0.j        0.129808-0.031153j  0.103847-0.000768j]
 [ 0.      +0.j       -0.087896-0.069165j -0.0700

OpenDSS Solution

In [9]:
dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
dss.run_command('Redirect 02node_threephase_unbalanced_unchanged.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect IEEE13_splitmultiphase.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced_edit.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')

VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')
kW_list = np.array([])
kvar_list = np.array([])

# Store the loads
for k in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k])
    kW_list = np.append(kW_list, dss.Loads.kW())
    kvar_list = np.append(kvar_list, dss.Loads.kvar())

for i in range(len(times)):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
    Vbase = dss.Bus.kVBase() * 1000
    Sbase = 1000000.0
    Ibase = Sbase/Vbase
    Zbase = Vbase/Ibase
    
    # Set slack bus (sourcebus) voltage reference in p.u.
    SlackBusVoltage = 1.000
    dss.Vsources.PU(SlackBusVoltage)
    
    # Time-varying load
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
        dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
      
    dss.Solution.Convergence(0.000000000001)
    
    # Solve power flow with OpenDSS file
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        # set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        # Easy process to get all names and count of loads, a trick to avoid
        # some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')
    
    
    for k in range(len(dss.Loads.AllNames())):
        dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0]) #bus name for load
        idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) #what index is bus at in allbusnames()
        load_ph_arr_temp = [0, 0, 0] #ph of load
        for ph1 in range(1, 4): #update existing ph of load
            pattern = r"\.%s" % (str(ph1))
            load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
            if load_ph:
                load_ph_arr_temp[ph1 - 1] = 1
        n= 0
        for m in range(len(load_ph_arr_temp)):
            if load_ph_arr_temp[m] == 1:
                #print('squirrel')
                load_arr[i, m, idxbs] +=  (dss.CktElement.Powers()[n] + 1j * dss.CktElement.Powers()[n + 1])*1e3/1e6     #where 1e6 is sbase 
                n += 2
    
    # print(dss.Solution.Converged())

    # Print number of buses, and bus names
    print(len(dss.Circuit.AllBusNames()))
    print(dss.Circuit.AllBusNames())

    # Print number of loads, and load names
    print(len(dss.Loads.AllNames()))
    print(dss.Loads.AllNames())

    print('')

    VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

    for k1 in range(len(dss.Circuit.AllBusNames())):
        
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
    #     print(dss.Circuit.AllBusNames()[k1])
    #     print(dss.Bus.Nodes())

    #     print('puVOTLAGES - LN CARTESIAN')
    #     print(dss.Bus.PuVoltage())

        ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
        
        Vtemp = np.asarray(dss.Bus.PuVoltage())

        Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
        

    #     print(np.asarray(dss.Bus.Nodes(),'int'))

        VDSS[ph,k1] = Vtemp    

    #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


    #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
    #     for k2 in range(len(dss.Bus.Nodes())):
    #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))


    IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
        Imn = np.asarray(dss.CktElement.Currents())/Ibase
    #     print(Imn)
        Imn = Imn[0:int(len(Imn)/2)]
    #     print(Imn)
        Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
    #     print(Imn)
        IDSS[ph,k1] = Imn
    #     print('')

    print('IDSS\n', np.round(IDSS,decimals=6))

    STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
    SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
    #     print(dss.CktElement.BusNames())
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    #     print(ph)
        Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
    #     print(Sk)

    #     print(Sk[0:int(len(Sk)/2)])
    #     print(Sk[int(len(Sk)/2):])

        STXtemp = Sk[0:int(len(Sk)/2)]
        SRXtemp = Sk[int(len(Sk)/2):]

        STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
    #     print(STXtemp)

        SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])
    #     print(SRXtemp)

        STXDSS[ph,k1] = STXtemp
        SRXDSS[ph,k1] = SRXtemp

    VDSS0[i, :, :] = VDSS
    IDSS0[i, :, :] = IDSS
    SRXDSS0[i, :, :] = SRXDSS
    STXDSS0[i, :, :] = STXDSS
    
    
    print('STXDSS\n', np.round(STXDSS,decimals=6))
    print('SRXDSS\n', np.round(SRXDSS,decimals=6))

    print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
    print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
    print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  11
Tolerance:  1e-12

3
['sourcebus', 'a01', 'a02']
6
['load_a01_a_01', 'load_a01_b_01', 'load_a01_c_01', 'load_a02_a_01', 'load_a02_b_01', 'load_a02_c_01']

VDSS
 [[ 0.999999-3.00000e-06j  0.982478+3.53260e-02j  0.970734+5.75090e-02j]
 [-0.500002-8.66023e-01j -0.466389-8.88240e-01j -0.447578-8.88603e-01j]
 [-0.499997+8.66026e-01j -0.535343+8.38345e-01j -0.551691+8.27287e-01j]]
IDSS
 [[ 0.230846-0.019907j  0.102695+0.005489j]
 [-0.141882-0.180067j -0.060711-0.110094j]
 [-0.054724+0.173832j -0.056054+0.083506j]]
STXDSS
 [[0.230846+0.019907j 0.10109 -0.001765j]
 [0.226884+0.032839j 0.126105+0.002579j]
 [0.177905+0.039523j 0.100015-0.002288j]]
SRXDSS
 [[0.22609 +0.028235j 0.1     +0.001087j]
 [0.226105+0.042579j 0.125   +0.005201j]
 [0.175015+0.047712j 0.1     +0.000225j]]
|VDSS|
 [[0.999999 0.983113 0.972436]
 [0.999999 1.003239 0.994958]
 [0.999999 0.994694 0.994368]]
<VDSS
 [[-1.87000000e-04 

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [10]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 
    print(np.max(np.abs(VDSS0[i] - VNR02[i])))

    print('Complex Current difference')
    print(np.max(np.abs(INR01[i] - INR02[i])))
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    print(np.max(np.abs(IDSS0[i] - INR02[i])))

    print('Complex TX Power difference')
    print(np.max(np.abs(STXNR01[i] - STXNR02[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR02[i])))

    print('Complex RX Power difference')
    print(np.max(np.abs(SRXNR01[i] - SRXNR02[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR02[i])))

    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VNR01[i]) - np.abs(VNR02[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR02[i]))))

    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VNR01[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print("\n")

Timestep 0 

Complex Voltage difference
2.220446049266469e-16
0.06458364706893287
0.06458364706893285
Complex Current difference
5.561946577567963e-17
0.012303364052816797
0.012303364052816814
Complex TX Power difference
5.561946577567963e-17
0.012304090868387563
0.012304090868387579
Complex RX Power difference
2.797157557069881e-17
0.002660647331595699
0.0026606473315957118
Voltage Magnitude difference
2.220446049250313e-16
0.027559151132557358
0.027559151132557247
Voltage Angle difference
1.6263032587282567e-19
3.6973432795445405
3.6973432795445405


Timestep 1 

Complex Voltage difference
1.2412670766236366e-16
0.06525065590268583
0.06525065590268583
Complex Current difference
3.469446951953614e-17
0.012535542432629308
0.01253554243262929
Complex TX Power difference
3.1031676915590914e-17
0.012536283872540887
0.012536283872540875
Complex RX Power difference
6.938893903907228e-17
0.0027076458463652685
0.002707645846365208
Voltage Magnitude difference
2.220446049250313e-16
0.027777907

Load comparison between non-vectorized and OpenDSS

In [7]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

Timestep 0 
 Magnitude of the Difference: 

[[0.00000000e+00 2.70384930e-11 6.42635962e-12 0.00000000e+00
  0.00000000e+00 5.32391906e-12 3.45714899e-11 0.00000000e+00
  9.79355018e-12 8.12026153e-13 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.36374656e-11 2.68471117e-12 3.77505624e-12
  2.39611839e-12 0.00000000e+00 2.81727322e-12 0.00000000e+00
  0.00000000e+00 1.66024492e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.01786324e-11 3.60895858e-12 0.00000000e+00
  3.11775754e-12 7.25589934e-12 1.72513228e-11 8.79027907e-12
  0.00000000e+00 4.14188786e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]]
Max difference: 
 
3.457148992939218e-11


Timestep 1 
 Magnitude of the Difference: 

[[0.00000000e+00 3.96133662e-13 9.40253225e-14 0.00000000e+00
  0.00000000e+00 6.72506085e-14 5.07442950e-13 0.00000000e+00
  1.43327653e-13 1.18911922e-14 0.00000000

---